# Meta-Regression {#metareg}

In [1]:
library(metaforest)

Loading required package: ggplot2

Loading required package: metafor

Loading required package: Matrix


Loading the 'metafor' package (version 3.0-2). For an
introduction to the package please type: help(metafor)


Loading required package: ranger

Loading required package: data.table



In [2]:
library(XLConnect)
df_attitude <- readWorksheetFromFile("/media/windowsC/SIN_Desktop/projects/2021/ENCHANT/datasets/ECHOES/WP4/ma_psycho_pred_esb.xls", sheet = 1)
df_intention <- readWorksheetFromFile("/media/windowsC/SIN_Desktop/projects/2021/ENCHANT/datasets/ECHOES/WP4/ma_psycho_pred_esb.xls", sheet = 2)
df_values <- readWorksheetFromFile("/media/windowsC/SIN_Desktop/projects/2021/ENCHANT/datasets/ECHOES/WP4/ma_psycho_pred_esb.xls", sheet = 3)
df_awareness <- readWorksheetFromFile("/media/windowsC/SIN_Desktop/projects/2021/ENCHANT/datasets/ECHOES/WP4/ma_psycho_pred_esb.xls", sheet = 4)
df_emotions <- readWorksheetFromFile("/media/windowsC/SIN_Desktop/projects/2021/ENCHANT/datasets/ECHOES/WP4/ma_psycho_pred_esb.xls", sheet = 5)

XLConnect 1.0.5 by Mirai Solutions GmbH [aut],
  Martin Studer [cre],
  The Apache Software Foundation [ctb, cph] (Apache POI),
  Graph Builder [ctb, cph] (Curvesapi Java library),
  Brett Woolridge [ctb, cph] (SparseBitSet Java library)

https://mirai-solutions.ch
https://github.com/miraisolutions/xlconnect



In [3]:
print(names(df_attitude))
print(names(df_intention))
print(names(df_values))
print(names(df_awareness))
print(names(df_emotions))

[1] "References" "n"          "r"          "llci"       "ulci"      
[1] "References" "n"          "r"          "llci"       "ulci"      
[1] "References" "n"          "r"          "llci"       "ulci"      
[1] "References" "n"          "r"          "llci"       "ulci"      
[1] "References" "n"          "r"          "llci"       "ulci"      


<br><br>

---

## Meta-regression in R

<br><br>

In [4]:
# Create function r_to_z, which takes r as input:
# Correlations are bounded by [-1, 1], they are often Fisher-transformed prior to meta-analysis
r_to_z <- function(r){.5 * log((1+r)/(1-r))}
v_z <- function(n){1/(n-3)}

In [5]:
df_attitude$zi <- r_to_z(df_attitude$r)
df_attitude$vi <- v_z(df_attitude$n)

df_intention$zi <- r_to_z(df_intention$r)
df_intention$vi <- v_z(df_intention$n)

df_values$zi <- r_to_z(df_values$r)
df_values$vi <- v_z(df_values$n)

df_awareness$zi <- r_to_z(df_awareness$r)
df_awareness$vi <- v_z(df_awareness$n)

df_emotions$zi <- r_to_z(df_emotions$r)
df_emotions$vi <- v_z(df_emotions$n)

<br><br>

---

## Plotting regressions

In [6]:
library(ggplot2)

<br></br>

---

In [7]:
library(repr)
# Change plot size to 4 x 3
options(repr.plot.width=12, repr.plot.height=9)

In [8]:
list_names_df <- list('attitude'=df_attitude, 
                'intention'=df_intention, 
                'values'=df_values, 
                'awareness'=df_awareness, 
                'emotions'=df_emotions)
fname <- sprintf("output/funnel.pdf")
#print(fname)
pdf(file=fname) # Open PDF device with specific file name
    
for (i in seq_along(list_names_df)){
    name <- names(list_names_df)[i]
    df <- list_names_df[[i]]
    
    m_re <- rma(yi = zi,
            vi = vi,
            data = df) # for publication bias funnel plot
    #print(names(m_re))
    #print(m_re)
    #print(m_re$beta[1])

    m_reg <- rma(yi = zi,
             vi = vi,
             mods = ~n,
             data = df) # for meta regression
    #print(m_reg)

    df$weights <- 1/sqrt(df$vi)
    # Specify basic plot, mapping sex to the x-axis, effect size 'd' to the y-axis,
    # and 'weights' to the weight parameter.
    
    PLOT_REG = TRUE #FALSE
    
    if (PLOT_REG) {
    reg_plot <- ggplot(df, aes(x = n, y = zi, size = weights)) +
      geom_point(shape = 1) + # Add scatter
      geom_abline(intercept = m_reg$b[1], slope = m_reg$b[2]) + # Add regression line
      theme_bw() + # Apply black and white theme
      theme(legend.position = "none") # Remove legend
      print(reg_plot)
    }
    funnel(m_re, level=c(90, 95, 99), shade=c("white", "gray55", "gray75"), refline=m_re$beta[1], 
           yaxis="sei", main=name)
    
}
dev.off() # Turn the PDF device off

Warning message:
“There are outcomes with non-positive sampling variances.”
Warning message:
“Negative sampling variances constrained to zero.”
Warning message:
“Cannot compute Q-test, I^2, or H^2 when there are non-positive sampling variances in the data.”
Warning message:
“There are outcomes with non-positive sampling variances.”
Warning message:
“Negative sampling variances constrained to zero.”
Warning message:
“Cannot compute QE-test, I^2, or H^2 when there are non-positive sampling variances in the data.”
Warning message in sqrt(df$vi):
“NaNs produced”
Warning message:
“Removed 1 rows containing missing values (geom_point).”


png 
  2